<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Larger_Context_Larger_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Packages and Setup Variables


In [ ]:
!pip install -q huggingface-hub==0.34.4 llama-index==0.14.0 openai==1.107.0 chromadb==1.0.21 jedi==0.19.2 \
                llama-index-llms-google-genai==0.5.0 llama-index-vector-stores-chroma==0.5.3 llama-index-embeddings-openai==0.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
# os.environ["GOOGLE_API_KEY"] = "[GOOGLE_API_KEY]"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Load Gemini Model

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.llms.google_genai import GoogleGenAI
import google.genai.types as types

config = types.GenerateContentConfig(thinking_config=types.ThinkingConfig(thinking_budget=0))

Settings.llm = GoogleGenAI(model="gemini-2.5-flash", generation_config=config)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

**Note: You can create a vector store from scratch using the code below, or you can load it from Hugging Face using the code provided in this notebook.**

## Vector Store

## Load the Dataset (JSON)

The dataset includes several articles from the TowardsAI blog, Research paper contents and Documentation which provide an in-depth explanation of AI models and RAG method.

In [ ]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

ai_tutor_knowledge.jsonl: 0.00B [00:00, ?B/s]

In [ ]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]
ai_tutor_knowledge[1]['content']

"Github Repo: https://github.com/vaibhawkhemka/ML-Umbrella/tree/main/NLP/Product-Categorization   From e-commerce to Customer support  all businesses require some kind of NER model to process huge amounts of texts from users.   To automate this whole  one requires NER models to extract relevant and important entities from text.   Final Result/OutputInput text = EL D68 (Green  32 GB) 3 GB RAM [3 GB RAM U+007C 32 GB ROM U+007C Expandable Upto 128 GB  15.46 cm (6.088 inch) Display  13MP Rear Camera U+007C 8MP Front Camera  4000 mAh Battery  Quad-Core Processor]   Output =   Green ->>>> COLOR 32 GB ->>>> STORAGE 3 GB RAM ->>>> RAM 3 GB RAM ->>>> RAM 32 GB ROM ->>>> STORAGE Expandable Upto 128 GB ->>>> EXPANDABLE_STORAGE 15.46 cm (6.088 inch) ->>>> SCREEN_SIZE 13MP Rear Camera ->>>> BACK_CAMERA 8MP Front Camera ->>>> FRONT_CAMERA 4000 mAh Battery ->>>> BATTERY_CAPACITY Quad-Core Processor ->>>> PROCESSOR_CORE   Data PreparationA tool for creating this dataset (https://github.com/tecoholic/n

In [ ]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)
doc[2]

Document(id_='45501b72-9391-529e-8e5e-59a2604ba26e', embedding=None, metadata={'url': 'https://towardsai.net/p/machine-learning/adaboost-explained-from-its-original-paper', 'title': 'AdaBoost Explained From Its Original Paper', 'tokens': 1697, 'source': 'tai_blog'}, excluded_embed_metadata_keys=['url', 'tokens', 'source'], excluded_llm_metadata_keys=['title', 'tokens', 'source'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="This publication is meant to show a very popular ML algorithm in complete detail  how it works  the math behind it  how to execute it in Python and an explanation of the proofs of the original paper. There will be math and code  but it is written in a way that allows you to decide which are the fun parts.   A bit on the origins of the algorithm: It was proposed by Yoav Freund and Robert E. Schapire in a 1997 paper  A Decision-Theoretic Generalization of On-Line Learning a

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

# Define the splitter object that split the text into segments with 1536 tokens,
# with a 128 overlap between the segments.
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.ingestion import IngestionPipeline

# set up ChromaVectorStore and load in data
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("ai_tutor_knowledge")

# save to disk
db = chromadb.PersistentClient(path="/content/ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
llm = OpenAI(model="gpt-5-mini", additional_kwargs={"reasoning_effort":"minimal"})

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=2, llm=llm),
        SummaryExtractor(summaries=["prev", "self"], llm=llm),
        KeywordExtractor(keywords=10, llm=llm),
        OpenAIEmbedding(model = "text-embedding-3-small"),
    ],
    vector_store=vector_store,
)

# Run the transformation pipeline.
nodes = pipeline.run(documents=doc, show_progress=True)


Generating embeddings:   100%|██████████| 5834/5834 [00:00<?, ?it/s]





In [ ]:
!zip -r vectorstore.zip ai_tutor_knowledge

# Download the Vector Store


You can Access the VectorStore from the Hugginface hub

In [ ]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip",repo_type="dataset",local_dir="/content")

!unzip vectorstore.zip

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

Archive:  vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# Load the vector store from the local storage.
db = chromadb.PersistentClient(path="/content/ai_tutor_knowledge")
chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
from llama_index.core import VectorStoreIndex

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [ ]:
for i in [2, 4, 6, 8, 10, 15, 20, 25, 30]:
    query_engine = index.as_query_engine(similarity_top_k=i)

    res = query_engine.query("Explain how RAG works?")

    print(f"top_{i} results:")
    print("\t", res.response)
    print("-_" * 20)

top_2 results:
	 Retrieval-Augmented Generation (RAG) models enhance language generation by combining pre-trained parametric and non-parametric memory. These models integrate a pre-trained sequence-to-sequence (seq2seq) model as their parametric memory with a dense vector index, such as one built from Wikipedia, which serves as non-parametric memory. A pre-trained neural retriever accesses this non-parametric memory.

The process involves retrieving relevant documents or passages, which are then passed to the seq2seq model to generate outputs. Both the retriever and the seq2seq modules are initialized from pre-trained models and fine-tuned jointly, allowing them to adapt to specific tasks. This architecture addresses challenges faced by Large Language Models (LLMs), such as outdated information and hallucinations, by retrieving relevant knowledge to improve accuracy, reliability, and trustworthiness in the generated language. RAG models have demonstrated effectiveness across various ta

top_6 results:
	 Retrieval-Augmented Generation (RAG) models combine pre-trained parametric and non-parametric memory for language generation, enhancing performance on knowledge-intensive natural language processing (NLP) tasks.

Here's a breakdown of how RAG works:

1.  **Integration of Pre-trained Models**: RAG models integrate a pre-trained dense retrieval (DPR) model with a pre-trained sequence-to-sequence (seq2seq) model. The seq2seq model acts as the parametric memory, while a dense vector index (e.g., of Wikipedia) serves as the non-parametric memory.

2.  **Retrieval Mechanism**: When a query or input is provided, the RAG model first uses a pre-trained neural retriever to access the non-parametric memory. It retrieves relevant documents or passages from the dense vector index.

3.  **Generation Mechanism**: The retrieved documents are then passed to the seq2seq model. The seq2seq model generates an output based on the input query and the context provided by the retrieved inform

top_10 results:
	 Retrieval-Augmented Generation (RAG) models combine pre-trained parametric and non-parametric memory for language generation. This process generally involves the following steps:

1.  **Retrieval Mechanism**: RAG models utilize a pre-trained neural retriever to access a dense vector index of knowledge, such as Wikipedia, which serves as non-parametric memory. This retriever fetches relevant documents based on a user's query or the context of the generated sequence.
2.  **Generation Mechanism**: The retrieved documents are then passed to a pre-trained sequence-to-sequence (seq2seq) model, which acts as the parametric memory. This model generates outputs by marginalizing over the retrieved documents.
3.  **Joint Fine-Tuning**: Both the retriever and the seq2seq modules are initialized from pre-trained models and are fine-tuned together. This joint fine-tuning allows both components to adapt and improve their performance on specific downstream tasks.

RAG models can oper

top_25 results:
	 Retrieval-Augmented Generation (RAG) models combine the strengths of pre-trained dense retrieval (DPR) and sequence-to-sequence (seq2seq) models to enhance language generation, particularly for knowledge-intensive NLP tasks.

Here's a breakdown of how RAG works:

1.  **Retrieval Mechanism**: When a query or input is provided, RAG models first retrieve relevant documents or passages from a dense vector index (e.g., a large corpus like Wikipedia) using a pre-trained neural retriever. This component acts as a non-parametric memory, allowing the model to access external knowledge.

2.  **Generation Mechanism**: The retrieved documents are then passed to a sequence-to-sequence model, which serves as the parametric memory. This seq2seq model generates outputs based on the information extracted from the retrieved documents.

3.  **Joint Fine-Tuning**: Both the retriever and the seq2seq modules are initialized from pre-trained models and fine-tuned together. This joint fine-t

top_30 results:
	 Retrieval-Augmented Generation (RAG) models combine the strengths of pre-trained dense retrieval (DPR) and sequence-to-sequence (seq2seq) models to enhance language generation, particularly in knowledge-intensive NLP tasks.

Here's a breakdown of how RAG works:

1.  **Retrieval Mechanism**: When a query or statement is given, the RAG model uses a pre-trained neural retriever to search for and retrieve relevant documents from a dense vector index, such as Wikipedia.

2.  **Generation Mechanism**: The retrieved documents, which serve as non-parametric memory, are then passed to a pre-trained seq2seq model, which acts as the parametric memory. This model uses the retrieved information to generate an output.

3.  **Joint Fine-Tuning**: Both the retriever and the seq2seq modules are initialized from pre-trained models and then fine-tuned together. This joint fine-tuning allows both the retrieval and generation components to adapt to specific downstream tasks, improving the

# Evaluate


In [ ]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/rag_eval_dataset.json

--2025-09-17 13:23:13--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/rag_eval_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 476714 (466K) [text/plain]
Saving to: ‘rag_eval_dataset.json’

rag_eval_dataset.js 100%[===================>] 465.54K  --.-KB/s    in 0.03s   

2025-09-17 13:23:13 (13.0 MB/s) - ‘rag_eval_dataset.json’ saved [476714/476714]



In [ ]:
# We can also load the dataset from a previously saved json file.
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json("./rag_eval_dataset.json")

In [ ]:
from llama_index.core.evaluation import (
    RelevancyEvaluator,
    FaithfulnessEvaluator,
    BatchEvalRunner,
)

from llama_index.llms.openai import OpenAI
llm_gpt5_mini = OpenAI(model="gpt-5-mini", additional_kwargs={"reasoning_effort":"minimal"})

faithfulness_evaluator = FaithfulnessEvaluator(llm=llm_gpt5_mini)
relevancy_evaluator = RelevancyEvaluator(llm=llm_gpt5_mini)

# Run evaluation
queries = list(rag_eval_dataset.queries.values())
batch_eval_queries = queries[:20]

runner = BatchEvalRunner(
    {"faithfulness": faithfulness_evaluator, "relevancy": relevancy_evaluator},
    workers=32,
)

for i in [2, 4, 6, 8, 10, 15, 20, 25, 30]:
    # Set Faithfulness and Relevancy evaluators
    query_engine = index.as_query_engine(similarity_top_k=i, llm=llm_gpt5_mini)

    eval_results = await runner.aevaluate_queries(
        query_engine, queries=batch_eval_queries
    )
    faithfulness_score = sum(
        result.passing for result in eval_results["faithfulness"]
    ) / len(eval_results["faithfulness"])
    print(f"top_{i} faithfulness_score: {faithfulness_score}")

    relevancy_score = sum(result.passing for result in eval_results["relevancy"]) / len(
        eval_results["relevancy"]
    )
    print(f"top_{i} relevancy_score: {relevancy_score}")

top_2 faithfulness_score: 1.0
top_2 relevancy_score: 1.0
top_4 faithfulness_score: 0.9
top_4 relevancy_score: 1.0
top_6 faithfulness_score: 0.95
top_6 relevancy_score: 1.0
top_8 faithfulness_score: 1.0
top_8 relevancy_score: 1.0
top_10 faithfulness_score: 1.0
top_10 relevancy_score: 1.0
top_15 faithfulness_score: 0.95
top_15 relevancy_score: 0.95
top_20 faithfulness_score: 0.9
top_20 relevancy_score: 1.0
top_25 faithfulness_score: 0.95
top_25 relevancy_score: 1.0
top_30 faithfulness_score: 0.85
top_30 relevancy_score: 1.0
